참고: https://hongl.tistory.com/245

가지고 있는 데이터의 형태는 이렇다 
- userid, movieid, rating, timestamp
- [1, 2, 3.5, 시간] 
- [1, 367, 3.5, 시간] 

이후 원핫인코딩을 이용하여 데이터를 전처리하게 됨 
- 따라서 (User의 수) + (item의 수) + (Rating) 만큼 열이 생긴다 
- -> 이전 데이터보다 훨씬 Sparse해짐 

In [1]:
import torch 
import torch.nn as nn 

In [ ]:
class FM(torch.nn.Module):
    def __init__(self, field_dimension, embeding_dimension):
        super().__init__()
        
        # nn.Embedding의 변수는 행렬곱을 의미, 작은 무작위 값으로 초기화 
        # nn.Parameter은 모델의 학습 가능한 매개변수를 나타냄, 0으로 초기화 
        self.embedding = nn.Embedding(sum(field_dimension), embeding_dimension)
        self.weighted_sum = nn.Embedding(sum(field_dimension), 1)
        self.global_bias = nn.Parameter(torch.zeros((1,)))

    def forward(self, x):
        # 시간 복잡도 Trick 사용 -> Linear하게 만들어줌 
        first = torch.sum(self.embedding(x), dim=1) ** 2
        second = torch.sum(self.embedding(x) ** 2, dim=1)
        
        # torch.sum에서 dim = 1은 행을 다 더하는 것 
        third_term  = torch.sum(0.5 * (first - second), dim = 1, keepdim=True)
        x = self.global_bias + torch.sum(self.weighted_sum(x), dim=1) + third_term

        # 크기가 1인 차원을 없애고, sigmoid를 씌움 
        return torch.sigmoid(x.squeeze(1))



In [ ]:
# field_dimension은 np.max(data, axis = 0) + 1로 구함 
# + 1은 Rating에 해당되는 열을 추가하기 위함 

'''
model = FM(field_dimension, 16)
criterion = torch.nn.BCELoss() 
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01, weight_decay=1e-5)
'''